In [1]:
import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('age_gender.csv')
df

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 149 147 145 146 147 148 149 149 150 153 153 153 152 153 153 153 153 153 151 149 147 146 146 144 143 140 134 129 129 126 122 121 119 118 118 99 53 130 129 128 126 127 130 135 141 143 143 144 145 143 142 144 145 146 149 149 148 151 151 151 151 152 153 153 153 153 149 148 147 146 146 142 141 138 133 130 130 128 123 124 122 120 117 98 52 131 129 130 129 129 134 138 144 145 143 143 139 140 138 139 140 142 145 147 149 151 149 149 149 151 152 151 150 150 148 147 145 144 142 140 137 136 135 134 131 128 126 126 123 122 114 97 50 135 134 133 132 131 136 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199 200 199 200 196 198 192 193 188 187 186 187 188 183 182 178 177 175 174 176 174 172 165 158 153 147 142 138 131 125 120 114 110 110 111 111 104 179 70 128 171 169 171 175 182 183 187 193 197 198 196 194 189 188 185 183 184 180 182 184 178 174 173 173 173 174 174 176 171 165 160 154 149 144 139 133 129 125 119 117 113 110 112 113 108 176 71 146 169 168 171 175 177 182 185 186 187 188 191 191 186 185 181 179 178 173 174 176 169 164 168 168 169 170 171 170 165 154 148 145 141 137 134 131 130 124 121 116 112 112 112 112 109 179 77 159 172 171 171 ...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155 161 166 169 175 177 178 179 180 183 186 187 188 192 194 198 203 206 213 214 216 220 219 215 213 211 211 210 207 206 203 200 197 194 192 65 66 69 70 70 69 72 83 88 100 112 125 139 155 159 165 167 170 174 176 178 177 180 185 186 187 190 193 195 197 201 207 208 212 215 215 215 213 211 211 210 207 204 201 200 197 194 192 61 63 69 71 70 70 72 81 87 100 113 123 137 149 153 158 164 166 171 173 173 174 182 185 184 188 190 191 189 192 197 201 204 210 213 212 211 211 208 209 208 206 204 199 199 197 194 190 56 62 69 71 72 72 73 80 88 103 114 124 134 147 ...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 211 211 211 209 205 204 204 204 205 205 207 207 207 207 208 208 208 208 207 207 205 204 203 202 202 202 205 209 214 221 226 227 224 223 234 243 250 193 197 198 200 200 202 203 205 206 207 207 208 209 210 209 207 206 206 206 207 207 207 207 207 207 208 208 208 208 207 207 205 204 203 202 204 203 205 209 214 219 224 227 224 223 234 243 250 196 197 198 201 201 204 205 206 206 206 207 208 210 209 209 208 207 206 206 207 207 208 208 209 209 209 209 208 208 206 205 205 205 203 203 203 203 205 208 213 217 223 226 224 224 234 243 250 197 199 198 201 203 ...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 219 220 221 221 220 221 221 221 220 220 222 222 222 220 220 218 218 218 217 217 218 218 219 219 218 220 222 219 223 225 221 210 169 117 154 214 227 203 207 209 210 209 209 209 211 211 212 216 217 218 219 219 220 221 221 221 222 222 222 222 222 220 220 218 218 218 217 217 218 218 217 217 217 219 222 218 222 223 221 218 179 113 122 195 222 205 206 207 209 209 209 210 211 212 213 216 216 217 218 219 220 221 222 222 222 222 222 222 221 221 219 219 218 218 217 216 218 218 217 216 216 219 221 219 223 223 222 222 186 121 104 172 213 206 208 207 209 209 ...
...,...,...,...,...,...
23700,99,0,1,20170120221920654.jpg.chip.jpg,127 100 94 81 77 77 74 99 102 98 128 145 160 177 187 180 189 198 181 179 181 191 198 204 204 207 199 203 208 202 202 195 176 173 159 149 137 108 91 100 106 116 151 144 128 111 106 101 120 101 82 80 73 68 73 89 97 111 140 154 157 171 182 169 191 205 211 210 206 204 207 209 206 209 209 213 216 204 210 213 205 201 195 192 177 151 96 102 98 118 144 137 122 110 100 96 124 110 85 72 73 77 89 105 106 121 122 157 180 188 191 189 205 219 227 230 229 227 219 212 210 218 226 222 225 206 193 201 191 200 210 200 196 181 128 100 88 122 137 135 122 105 90 90 124 114 99 78 72 82 95 115 120 144 173 185 183...
23701,99,1,1,20170120134639935.jpg.chip.jpg,23 28 32 35 42 47 68 85 98 103 113 117 130 129 128 139 

In [3]:
def process_pixels(pixels):
    pixels = np.array(pixels.split(), dtype='float32')
    return pixels.reshape((48, 48))

In [4]:
df['image'] = df['pixels'].apply(process_pixels)

train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
def create_image(row):
    return PILImage.create((row['image'] * 255).astype(np.uint8))

train_df['image_pil'] = train_df.apply(create_image, axis=1)
valid_df['image_pil'] = valid_df.apply(create_image, axis=1)

In [6]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=ColReader('image_pil'),
    get_y=ColReader('ethnicity'),
    splitter=RandomSplitter(valid_pct=0.2),
    item_tfms=Resize(224),  
    batch_tfms=[*aug_transforms(mult=2.), Normalize.from_stats(*imagenet_stats)]
)

In [7]:
dataloaders = dblock.dataloaders(train_df, bs=32)

In [8]:
learn = vision_learner(dataloaders, resnet34, metrics=accuracy)

if torch.cuda.is_available():
    learn.model.cuda()

learn.fine_tune(10, base_lr=3e-3) 

epoch,train_loss,valid_loss,accuracy,time
0,1.397028,1.183531,0.536129,01:14


epoch,train_loss,valid_loss,accuracy,time
0,0.966641,0.755983,0.735496,01:45
1,0.805038,0.697532,0.773207,01:45
2,0.808927,0.719562,0.734968,01:45
3,0.703596,0.643779,0.782173,01:45
4,0.644532,0.642403,0.780063,01:45
5,0.600197,0.530117,0.819884,01:45
6,0.536548,0.504486,0.832542,01:46
7,0.482882,0.489992,0.839926,01:46
8,0.438078,0.478141,0.841245,01:45
9,0.423719,0.486249,0.840717,01:45


In [9]:
def get_ethnicity_probabilities(image):
    pred, pred_idx, probs = learn.predict(image)
    return {learn.dls.vocab[i]: p.item() for i, p in enumerate(probs)}

In [10]:
example_image = train_df['image_pil'].iloc[0]
probabilities = get_ethnicity_probabilities(example_image)
print(probabilities)

{0: 0.9980795383453369, 1: 0.0015875262906774879, 2: 2.329841117898468e-05, 3: 0.00015570560935884714, 4: 0.0001538845244795084}


In [11]:
df = df[df['age'] >= 45]
df

,age,ethnicity,gender,img_name,pixels,image
1279,100,0,0,20170112213500903.jpg.chip.jpg,203 185 139 163 176 184 186 175 175 156 159 136 115 125 123 108 115 120 118 111 111 118 115 121 122 119 119 114 116 122 124 123 131 142 159 182 181 193 198 134 151 148 125 150 121 100 95 95 204 149 135 164 184 191 176 173 146 136 169 155 146 148 135 130 135 125 135 137 134 117 112 119 119 119 127 115 118 118 122 125 133 139 165 166 198 180 200 122 147 139 132 121 149 108 100 96 195 164 153 163 185 203 179 151 191 151 150 143 154 136 133 133 125 129 150 149 142 139 133 121 130 123 122 133 132 131 128 123 105 135 154 175 189 199 198 148 159 157 146 118 123 142 113 102 185 170 165 169 177 186...,"[[203.0, 185.0, 139.0, 163.0, 176.0, 184.0, 186.0, 175.0, 175.0, 156.0, 159.0, 136.0, 115.0, 125.0, 123.0, 108.0, 115.0, 120.0, 118.0, 111.0, 111.0, 118.0, 115.0, 121.0, 122.0, 119.0, 119.0, 114.0, 116.0, 122.0, 124.0, 123.0, 131.0, 142.0, 159.0, 182.0, 181.0, 193.0, 198.0, 134.0, 151.0, 148.0, 125.0, 150.0, 121.0, 100.0, 95.0, 95.0], [204.0, 149.0, 135.0, 164.0, 184.0, 191.0, 176.0, 173.0, 146.0, 136.0, 169.0, 155.0, 146.0, 148.0, 135.0, 130.0, 135.0, 125.0, 135.0, 137.0, 134.0, 117.0, 112.0, 119.0, 119.0, 119.0, 127.0, 115.0, 118.0, 118.0, 122.0, 125.0, 133.0, 139.0, 165.0, 166.0, 198.0,..."
1280,100,0,0,20170112215240346.jpg.chip.jpg,125 126 115 78 66 66 77 85 95 102 117 138 138 146 170 177 184 198 196 187 199 214 220 222 232 234 236 232 234 237 241 230 227 220 227 217 214 205 210 201 188 202 192 161 163 160 157 157 124 122 94 79 68 65 80 96 86 105 118 129 146 167 175 177 177 190 177 184 212 217 224 225 232 234 232 228 231 233 229 226 227 229 218 226 213 202 207 202 195 201 205 179 167 163 161 157 126 110 91 71 55 62 77 96 90 99 118 131 135 165 177 171 183 188 183 197 209 217 220 229 230 235 238 232 234 236 232 227 223 224 222 221 212 219 205 203 208 187 205 199 169 164 161 159 124 93 76 65 60 65 80 94 93 105 115 129 1...,"[[125.0, 126.0, 115.0, 78.0, 66.0, 66.0, 77.0, 85.0, 95.0, 102.0, 117.0, 138.0, 138.0, 146.0, 170.0, 177.0, 184.0, 198.0, 196.0, 187.0, 199.0, 214.0, 220.0, 222.0, 232.0, 234.0, 236.0, 232.0, 234.0, 237.0, 241.0, 230.0, 227.0, 220.0, 227.0, 217.0, 214.0, 205.0, 210.0, 201.0, 188.0, 202.0, 192.0, 161.0, 163.0, 160.0, 157.0, 157.0], [124.0, 122.0, 94.0, 79.0, 68.0, 65.0, 80.0, 96.0, 86.0, 105.0, 118.0, 129.0, 146.0, 167.0, 175.0, 177.0, 177.0, 190.0, 177.0, 184.0, 212.0, 217.0, 224.0, 225.0, 232.0, 234.0, 232.0, 228.0, 231.0, 233.0, 229.0, 226.0, 227.0, 229.0, 218.0, 226.0, 213.0, 202.0, 207..."
1281,100,0,1,20170110183726390.jpg.chip.jpg,235 164 113 144 121 201 243 237 234 221 233 235 232 233 236 234 215 226 199 211 204 185 213 179 210 188 195 211 198 175 167 176 151 198 172 114 130 123 130 126 125 126 60 24 35 28 89 227 233 107 119 86 45 239 239 236 235 236 232 233 231 231 222 223 213 224 215 217 219 201 217 200 199 186 188 209 203 183 180 166 134 165 176 156 145 131 128 116 136 106 112 15 30 38 27 225 197 109 104 113 138 242 234 234 236 228 229 228 212 227 229 226 225 224 227 211 218 211 211 217 211 209 186 220 195 187 167 155 140 162 146 151 154 143 115 111 120 124 106 43 19 40 18 223 124 104 76 50 231 244 234 234 238 2...,"[[235.0, 164.0, 113.0, 144.0, 121.0, 201.0, 243.0, 237.0, 234.0, 221.0, 233.0, 235.0, 232.0, 233.0, 236.0, 234.0, 215.0, 226.0, 199.0, 211.0, 204.0, 185.0, 213.0, 179.0, 210.0, 188.0, 195.0, 211.0, 198.0, 175.0, 167.0, 176.0, 151.0, 198.0, 172.0, 114.0, 130.0, 123.0, 130.0, 126.0, 125.0, 126.0, 60.0, 24.0, 35.0, 28.0, 89.0, 227.0], [233.0, 107.0, 119.0, 86.0, 45.0, 239.0, 239.0, 236.0, 235.0, 236.0, 232.0, 233.0, 231.0, 231.0, 222.0, 223.0, 213.0, 224.0, 215.0, 217.0, 219.0, 201.0, 217.0, 200.0, 199.0, 186.0, 188.0, 209.0, 203.0, 183.0, 180.0, 166.0, 134.0, 165.0, 176.0, 156.0, 145.0, 131...."
1282,100,0,1,20170112213001988.jpg.chip.jpg,82 82 82 82 82 82 84 86 90 85 104 105 102 154 182 193 211 215 202 197 184 199 200 194 186 190 202 205 194 191 196 188 176 166 159 160 157 149 157 128 139 131 149 142 139 158 

In [12]:
example_image = df['image'].iloc[0]
print(get_ethnicity_probabilities(example_image))

{0: 0.46442803740501404, 1: 0.5133423805236816, 2: 0.003164557972922921, 3: 0.014192746952176094, 4: 0.004872326273471117}


In [13]:
learn.export('./models/fastai_model_v1.pkl')

In [14]:
benchmark = learn.validate()
print(f'Accuracy: {benchmark[1]:.4f}')

Accuracy: 0.8407
